In [1]:
import numpy as np
import pandas as pd
from mlcbakery import bakery_client
import pathlib
client = bakery_client.Client("http://localhost:8000", token="authtoken")

def add_rand_column_to_dataset(path_to_csv, column_name):
    df = pd.read_csv(path_to_csv)
    df[column_name] = np.random.randint(0, 100)
    df.to_csv(path_to_csv, index=False)


In [4]:
client.find_or_create_by_collection_name("test23")

BakeryCollection(id=3995, name='test23', description='', storage_info=None, storage_provider=None)

In [7]:
attributed_to = "jon@jetty.io"
# prepares the dataset folder for the bakery by creating a .bakery.json file based on the folder structure
client.prepare_dataset("./datasets/A1_000", params={
    "properties": {
        "type": "dataset",
        "name": "A1_000",
        "collection_name": "test23",
        "origin": "jetty.io",
        "metadata_version": "1.0.0",
    },
    "parents": [
        {
            "generated_by": None,
            "attributed_to": attributed_to
        }
    ],
    "assets": {
        "long_description": "README.md",
        "metadata": "metadata.json"
    }    
})

2025-05-11 03:19:08,874 - mlcbakery.bakery_client - INFO - Created .bakery.json in './datasets/A1_000'


{'properties': {'type': 'dataset',
  'name': 'A1_000',
  'collection_name': 'test23',
  'origin': 'jetty.io',
  'metadata_version': '1.0.0'},
 'parents': [{'generated_by': None, 'attributed_to': 'jon@jetty.io'}],
 'assets': {'long_description': 'README.md', 'metadata': 'metadata.json'}}

In [30]:
import json
# load json:
with open("../../bakerydev-bakery-storage-sa.json", "r") as f:
    storage_info = json.load(f)
client.update_collection_storage_info("test23", storage_info=storage_info, storage_provider="gcp")
get_storage_info = client.get_collection_storage_info("test23")

In [32]:
ds = client.save_to_bakery("./datasets/A1_000", upload_data=True)
ds

2025-05-11 03:24:27,838 - absl - WARNING - WARNING: The JSON-LD `@context` is not standard. Refer to the official @context (e.g., from the example datasets in https://github.com/mlcommons/croissant/tree/main/datasets/1.0). The different keys are: {'includes', 'md5', 'dct', 'fileObject', 'conformsTo', 'jsonPath', 'isLiveDataset', 'transform', 'fileProperty', 'parentField', 'cr', 'subField', 'data', 'path', 'fileSet', 'rai', 'format', 'key', 'repeated', 'replace', 'references', 'separator', 'examples', '@vocab', 'regex', '@language', 'citeAs'}
2025-05-11 03:24:27,856 - mlcbakery.bakery_client - INFO - Created tar.gz of data folder at /tmp/tmpgn24j6_j.tar.gz
2025-05-11 03:24:27,857 - mlcbakery.bakery_client - INFO - Pushing dataset 'test23/A1_000' to Bakery API
2025-05-11 03:24:27,899 - absl - WARNING - WARNING: The JSON-LD `@context` is not standard. Refer to the official @context (e.g., from the example datasets in https://github.com/mlcommons/croissant/tree/main/datasets/1.0). The diff

BakeryDataset(id=950, name='A1_000', collection_id=3995, parent_collection_dataset=None, metadata=Dataset(jsonld={'@context': {'sc': 'https://schema.org/', 'name': 'sc:name', 'description': 'sc:description', 'license': 'sc:license', 'distribution': 'sc:distribution', 'recordSet': 'sc:recordSet', 'field': 'sc:field', 'dataType': 'sc:dataType', 'source': 'sc:source', 'extract': 'sc:extract', 'column': 'sc:column', 'contentUrl': 'sc:contentUrl', 'encodingFormat': 'sc:encodingFormat', 'citation': 'sc:citation', 'datePublished': 'sc:datePublished', 'version': 'sc:version', '@base': 'cr_base_iri/'}, '@type': 'sc:Dataset', 'name': 'A1_000', 'description': 'Country population and favourite food dataset.', 'license': 'CC-BY-4.0', 'citation': 'Source: Example dataset for Jetty/MLC Bakery demo.', 'datePublished': '2024-06-07', 'version': '1.0.0', 'distribution': [{'@type': 'https://schema.org/FileObject', 'name': 'data.csv', 'contentUrl': 'data/data.csv', 'encodingFormat': 'text/csv', 'https://sc

In [38]:
# client.duplicate_dataset("datasets/A1_000", "datasets/B1_000", params={
#     "properties": {
#         "name": "B1_000"
#     },
# }, attributed_to="jon@jetty.io")
# add_rand_column_to_dataset("datasets/B1_000/data/data.csv", "B1")
ds = client.save_to_bakery("./datasets/B1_000", upload_data=True)

2025-05-11 03:26:21,925 - absl - WARNING - WARNING: The JSON-LD `@context` is not standard. Refer to the official @context (e.g., from the example datasets in https://github.com/mlcommons/croissant/tree/main/datasets/1.0). The different keys are: {'includes', 'md5', 'dct', 'fileObject', 'conformsTo', 'jsonPath', 'isLiveDataset', 'transform', 'fileProperty', 'parentField', 'cr', 'subField', 'data', 'path', 'fileSet', 'rai', 'format', 'key', 'repeated', 'replace', 'references', 'separator', 'examples', '@vocab', 'regex', '@language', 'citeAs'}
2025-05-11 03:26:21,935 - mlcbakery.bakery_client - INFO - Created tar.gz of data folder at /tmp/tmpggav21t5.tar.gz
2025-05-11 03:26:21,935 - mlcbakery.bakery_client - INFO - Pushing dataset 'test23/B1_000' to Bakery API
2025-05-11 03:26:21,959 - mlcbakery.bakery_client - ERROR - Request failed: 404 Client Error: Not Found for url: http://localhost:8000/api/v1/datasets/test23/B1_000
2025-05-11 03:26:21,960 - mlcbakery.bakery_client - INFO - Dataset

In [ ]:
# client.push_dataset("notebooks/datasets/A1_000")

# client.duplicate_dataset("notebooks/datasets/A1_000", "notebooks/datasets/B1_000", params={
#     "properties": {
#         "name": "B1_000"
#     },
# }, attributed_to=attributed_to)
# add_rand_column_to_dataset("notebooks/datasets/B1_000/data.csv", "B1")
# client.push_dataset("notebooks/datasets/B1_000")

In [1]:
# take dataset A_### 
# generate some metadata
# save the metadata + dataset as A1_###

# grab a dataset A1_###
# change the dataset + metadata
# save the metadata + dataset as B1_###

# grab a dataset A1_###
# change the dataset + metadata
# save the metadata + dataset as C1_###


# grab a dataset B1_###
# change the dataset + metadata
# save the metadata + dataset as D1_###




2